In [1]:
import AlGDock.BindingPMF_plots
from AlGDock.BindingPMF_plots import *
import os, shutil, glob

phases = ['NAMD_Gas', 'OpenMM_OBC2']
# 'NAMD_OBC', 
# phases = ['sander_Gas', \
#   'sander_HCT', 'sander_OBC1', 'sander_OBC2', 'sander_GBn', 'sander_GBn2', \
#   'sander_PBSA', 'sander_ALPB_HCT', 'sander_ALPB_OBC1', 'sander_ALPB_OBC2', \
#   'sander_ALPB_GBn', 'gbnsr6_Still', 'gbnsr6_CHA']

self = AlGDock.BindingPMF_plots.BPMF_plots(\
  dir_CD='CD', dir_BC='BC',\
  ligand_database='prmtopcrd/ligand.db', \
  forcefield='prmtopcrd/gaff2.dat', \
  ligand_prmtop='prmtopcrd/ligand.prmtop', \
  ligand_inpcrd='prmtopcrd/ligand.trans.inpcrd', \
  ligand_mol2='prmtopcrd/ligand.mol2', \
  ligand_rb='prmtopcrd/ligand.rb', \
  receptor_database='prmtopcrd/receptor.db', \
  receptor_prmtop='prmtopcrd/receptor.prmtop', \
  receptor_inpcrd='prmtopcrd/receptor.trans.inpcrd', \
  receptor_fixed_atoms='prmtopcrd/receptor.pdb', \
  complex_prmtop='prmtopcrd/complex.prmtop', \
  complex_inpcrd='prmtopcrd/complex.trans.inpcrd', \
  complex_fixed_atoms='prmtopcrd/complex.pdb', \
  score = 'prmtopcrd/xtal_plus_dock6_scored.mol2', \
  temperature_scaling = 'Quadratic', \
  pose = -1, \
  rmsd=True, \
  dir_grid='grids', \
  protocol='Adaptive', BC_therm_speed=25.0, CD_therm_speed=0.25, \
  T_HIGH=450.0, T_SIMMIN=300.0, T_TARGET=300.0, \
  sampler='HMC', \
  MCMC_moves=1, \
  sampling_importance_resampling = True, \
  solvation = 'Full', \
  seeds_per_state=10, steps_per_seed=200, darts_per_seed=0, \
  sweeps_per_cycle=50, snaps_per_cycle=25, attempts_per_sweep=100, \
  steps_per_sweep=50, darts_per_sweep=0, \
  BC_repX_cycles=3, CD_repX_cycles=4, \
  site='Sphere', site_center=[1.7416, 1.7416, 1.7416], \
  site_max_R=1.0, \
  site_density=10., \
  phases=phases, \
  cores=-1, \
  random_seed=-1, \
  max_time=240, \
  keep_intermediate=True)

using 12/12 available cores
using random number seed of -1
  no progress information for CD
  no progress information for BC

*** Directories ***
  start: /Users/dminh/Documents/Code/AlGDock-0.0.1/Example
  BC: /Users/dminh/Documents/Code/AlGDock-0.0.1/Example/BC
  CD: /Users/dminh/Documents/Code/AlGDock-0.0.1/Example/CD

*** Files ***
  ligand_database: prmtopcrd/ligand.db
  receptor_database: prmtopcrd/receptor.db
  forcefield: prmtopcrd/gaff2.dat
  frcmodList: ['/Users/dminh/Documents/Code/AlGDock-0.0.1/Example/prmtopcrd/ligand.frcmod']
  prmtop:
    L: prmtopcrd/ligand.prmtop
    R: prmtopcrd/receptor.prmtop
    RL: prmtopcrd/complex.prmtop
  inpcrd:
    L: prmtopcrd/ligand.trans.inpcrd
    R: prmtopcrd/receptor.trans.inpcrd
    RL: prmtopcrd/complex.trans.inpcrd
  mol2:
    L: prmtopcrd/ligand.mol2
  fixed_atoms:
    R: prmtopcrd/receptor.pdb
    RL: prmtopcrd/complex.pdb
  grids:
    LJr: grids/LJr.nc
    LJa: grids/LJa.nc
    sELE: grids/pbsa.nc
    ELE: grids/direct_ELE.nc
    

In [ ]:
self.log.clear_lock('BC')
self.log.clear_lock('CD')
self._run('all')


>>> Ligand preparation for BC, starting at Thu, 17 Oct 2019 17:11:46
  minimized to -301 kcal/mol over 10 steps
  T = 20, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.812
  T = 21, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.832
  T = 24, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.844
  T = 26, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.828
  T = 29, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.836
  T = 31, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.872
  T = 35, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.836
  T = 38, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.856
  T = 42, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.852
  T = 46, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.868
  T = 50, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.852
  T = 55, delta_t = 2.000 fs, steps_per_trial = 10, acc_rate = 0.832
  T = 61, delta_t = 2.000 fs, steps_per_trial = 10, acc_rat

/miniconda2/envs/algdock/lib/python2.7/site-packages/scipy/optimize/minpack.py:237: RuntimeWarning: xtol=0.000000 is too small, no further improvement in the approximate
  solution is possible.
  warnings.warn(msg, RuntimeWarning)


  completed cycle 2 in 3.74 s
  Sampler 346/650=0.53 (4.2 s); repX t 0.5 s
  wrote to BC_progress.pkl.gz
  saved BC progress
  wrote to BC_data.pkl.gz
  saved BC data

  calculated BC free energy of -26.62 RT using cycles 1 to 2
Elapsed time for 2 cycles of replica exchange: 7.83 s

>>> Postprocessing, starting at Thu, 17 Oct 2019 17:12:01

  postprocessed BC, state 12, cycle 0, LNAMD_Gas in 0.16 s
  postprocessed BC, state 12, cycle 1, LNAMD_Gas in 0.15 s
  postprocessed BC, state 12, cycle 2, LNAMD_Gas in 0.16 s
